# STEPS TO FOLLOW
### Iterate through each entity (except EPUK and EPL) and for each currency
### 1) Hedge between above and below for each entity != entity_ccy using the entity_ccy as the back-to-back
### 2) Add those amounts of entity_ccy to the corresponding above/below of the entity ccy
### 3) Move the excess of each currency (except entity_ccy) from the entity to EPUK, using the entity_ccy as back-to-back
### 4) Add the amount of these movements of entity_ccy to the entity's exposure in the entity_ccy in the above/below (for each entity and EPUK)
### Now, all the currencies for each entity are hedged, except for the entity_ccy. The entity_ccy exposure isn't moved to EPUK
### 5) Now, for EPUK, hedge between above and below for each entity != entity_ccy using the entity_ccy as the back to back
### 6) Hedge the excess of each currency != entity_ccy in the market with EPUK
### 7) For each entity_ccy in each entity, hedge the excess to the entity_threshold with EPUK (without moving the exposure to EPUK)
### 8) These last hedges need to be moved to EPL so EPUK isn't exposed to that currency, so last step is to do a internal EPUK<->EPL

In [1]:
import pandas as pd
import numpy as np
import queries as qy

In [2]:
import os
from configparser import ConfigParser

def get_config():
    config = ConfigParser()
    config.read(os.path.join("C:/Users/andres.mireles_ebury/Desktop/Projects/FX Exposure/fx_exposure/config.ini"))

    return config

from bq_link import get_bq_link
bq_client, _ = get_bq_link(get_config())

Load all the exposures

In [3]:
balance_date = "2024-04-30"
exposures_entities = bq_client.query(qy.net_exposure.format(date=balance_date)).drop_duplicates()

Set the group threshold and the individual entity threshold (in the future, these can be weighted)

The total exposures by currency

In [4]:
total_exposures = exposures_entities.groupby("currency")[["above_exposure_gbp","below_exposure_gbp","net_exposure_gbp"]].sum()

Use only the entitiies with high exposures

In [5]:
group_threshold = 5e6

In [6]:
large_exposure_currencies = total_exposures[
    (np.abs(total_exposures.above_exposure_gbp)>=group_threshold)|
    (np.abs(total_exposures.below_exposure_gbp)>=group_threshold)|
    (np.abs(total_exposures.net_exposure_gbp)>=group_threshold)
].index

In [7]:
large_exposure_currencies

Index(['AED', 'AUD', 'CAD', 'CHF', 'EUR', 'GBP', 'HKD', 'PLN', 'USD'], dtype='object', name='currency')

In [8]:
exposures_entities = exposures_entities[exposures_entities.currency.isin(large_exposure_currencies)].reset_index(drop=True)

In [9]:
exposures_entities = exposures_entities.set_index(["currency","entity","entity_ccy"])

# THIS PART IS NOT CORRECT (IT DOESN'T ALLOW US TO SEE THE FULL EXPOSURE) AND NOT USEFUL

Only the entities with large exposures to each currency

In [10]:
entity_threshold = group_threshold / (2 * exposures_entities.index.get_level_values(1).unique().shape[0])

In [11]:
# large_exposure_entities_currencies = exposures_entities[
#     (np.abs(exposures_entities.above_exposure_gbp)>=entity_threshold)|
#     (np.abs(exposures_entities.below_exposure_gbp)>=entity_threshold)|
#     (np.abs(exposures_entities.net_exposure_gbp)>=entity_threshold)
# ].index

In [12]:
# exposures_entities = exposures_entities.loc[large_exposure_entities_currencies]

In [13]:
def net_exposure(currency,entity,entity_ccy, above,above_local_ccy,above_entity_ccy,below,below_local_ccy,below_entity_ccy,threshold,exclude_epuk=True,exclude_epl=True):

    if above == below == 0 or (exclude_epl and entity == "EPL") or (exclude_epuk and entity == "EPUK") or currency == entity_ccy:
        return above, above_local_ccy, above_entity_ccy, below, below_local_ccy, below_entity_ccy, above+below, above_local_ccy+below_local_ccy, above_entity_ccy+below_entity_ccy, 0, 0, 0, np.nan, np.nan, 0, 0, 0, np.nan
    

    # Rate to convert gbp to local ccy
    if below != 0:
        rate_to_local_ccy = below_local_ccy / below
        rate_to_entity_ccy = below_entity_ccy / below
    else:
        rate_to_local_ccy = above_local_ccy / above
        rate_to_entity_ccy = above_entity_ccy / above


    # Signs of the below to move the exposure (if below is 0, then just the opposite of the sign of the above)
    if below != 0:
        sign_below = below / abs(below)
    else:
        sign_below = -1 * above / abs(above)

    # The internal change that we need
    internal_change = np.maximum(abs(below) - threshold,0)

    internal_change_local_ccy = internal_change * rate_to_local_ccy
    internal_change_entity_ccy = internal_change * rate_to_entity_ccy

    # The resulting exposures
    new_above = above + sign_below*internal_change
    new_above_local_ccy = above_local_ccy + sign_below*internal_change_local_ccy
    new_above_entity_ccy = above_entity_ccy + sign_below*internal_change_entity_ccy
    new_below = below - sign_below*internal_change
    new_below_local_ccy = below_local_ccy - sign_below*internal_change_local_ccy
    new_below_entity_ccy = below_entity_ccy - sign_below*internal_change_entity_ccy

    # The direction of the internal entity hedges
    below_lhs_rhs = np.where(
        internal_change != 0,
        np.where(sign_below>0,"LHS","RHS"),
        np.nan
    )
    above_lhs_rhs = np.where(
        internal_change != 0,
        np.where(sign_below>0,"RHS","LHS"),
        np.nan
    )

    # What we are going to move from the above of the entity to the above in epuk
    above_to_epuk = np.maximum(abs(new_above) - threshold,0)

    above_to_epuk_local_ccy = above_to_epuk * rate_to_local_ccy
    above_to_epuk_entity_ccy = above_to_epuk * rate_to_entity_ccy
    
    # The direction of the above to epuk hedges (if above = 0, we don't move anything)
    above_to_epuk_lhs_rhs = np.where(
        above_to_epuk != 0,
        np.where(new_above/abs(new_above)>0,"LHS","RHS"),
        np.nan
    )

    # Update the new above of that entity reducing what we move to epuk
    new_above = new_above - np.where(
        new_above!=0,
        new_above/abs(new_above)*above_to_epuk,
        0
    )
    new_above_local_ccy = new_above_local_ccy - np.where(
        new_above!=0,
        new_above_local_ccy/abs(new_above_local_ccy)*above_to_epuk_local_ccy,
        0
    )
    new_above_entity_ccy = new_above_entity_ccy - np.where(
        new_above!=0,
        new_above_entity_ccy/abs(new_above_entity_ccy)*above_to_epuk_entity_ccy,
        0
    )

    # New net
    new_net = new_below + new_above
    new_net_local_ccy = new_below_local_ccy + new_above_local_ccy
    new_net_entity_ccy = new_below_entity_ccy + new_above_entity_ccy
    


    return new_above, new_above_local_ccy, new_above_entity_ccy, new_below, new_below_local_ccy, new_below_entity_ccy, new_net, new_net_local_ccy, new_net_entity_ccy, internal_change, internal_change_local_ccy, internal_change_entity_ccy, above_lhs_rhs, below_lhs_rhs, above_to_epuk, above_to_epuk_local_ccy, above_to_epuk_entity_ccy, above_to_epuk_lhs_rhs


In [14]:
exposures_entities.head()

,,,above_exposure_gbp,above_exposure_local_ccy,above_exposure_entity_ccy,below_exposure_gbp,below_exposure_local_ccy,below_exposure_entity_ccy,net_exposure_gbp,net_exposure_local_ccy,net_exposure_entity_ccy
currency,entity,entity_ccy,,,,,,,,,
EUR,EPDIFC,USD,1334502.38,1562831.78,1.669729e+06,2.174677e+05,254675.78,2.720956e+05,1.551970e+06,1817507.56,1.941825e+06
CHF,EPUK,GBP,-4355491.77,-5000000.00,-4.355492e+06,3.180092e+06,3650669.25,3.180092e+06,-1.175400e+06,-1349330.75,-1.175400e+06
PLN,EPAU,AUD,0.00,0.00,0.000000e+00,-8.745367e+01,-443.11,-1.686248e+02,-8.745367e+01,-443.11,-1.686248e+02
GBP,EPHK,HKD,11136.29,11136.29,1.089778e+05,7.287082e+06,7287082.44,7.131010e+07,7.298219e+06,7298218.73,7.141907e+07
CHF,EPDIFC,USD,0.00,0.00,0.000000e+00,7.553381e+02,867.11,9.450790e+02,7.553381e+02,867.11,9.450790e+02


## Step 1)

### For each entity except EPUK & EPL, we hedge above/below and move out of each ccy except entity_ccy the remaining exposure

In [15]:
import warnings

warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

exposures_entities_net = pd.DataFrame()

for entity in exposures_entities.index.get_level_values(1).unique():
    
    exposure_ent = exposures_entities.xs(entity, level=1)

    exposure_ent[
        [
            'new_above_exposure_gbp', 
            'new_above_exposure_local_ccy', 
            'new_above_exposure_entity_ccy', 
            'new_below_exposure_gbp', 
            'new_below_exposure_local_ccy', 
            'new_below_exposure_entity_ccy', 
            'new_net_exposure_gbp', 
            'new_net_exposure_local_ccy', 
            'new_net_exposure_entity_ccy', 
            'internal_change_gbp', 
            'internal_change_local_ccy', 
            'internal_change_entity_ccy', 
            'above_lhs_rhs', 
            'below_lhs_rhs', 
            'above_to_epuk_gbp', 
            'above_to_epuk_local_ccy', 
            'above_to_epuk_entity_ccy', 
            'above_to_epuk_lhs_rhs'
        ]
    ] = exposure_ent.apply(
        lambda row: pd.Series(
            net_exposure(
                row.name[0],
                entity, 
                row.name[1], 
                row['above_exposure_gbp'], 
                row['above_exposure_local_ccy'], 
                row['above_exposure_entity_ccy'], 
                row['below_exposure_gbp'], 
                row['below_exposure_local_ccy'], 
                row['below_exposure_entity_ccy'], 
                entity_threshold,
                exclude_epuk=True,
                exclude_epl=True
            )
        ),
        axis=1
    )

    exposure_ent["entity"] = entity
    exposure_ent = exposure_ent.reset_index().set_index(["currency","entity","entity_ccy"])
    
    exposures_entities_net = pd.concat([exposures_entities_net,exposure_ent])
    

C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:72: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\andres.mireles_ebury\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in double_scalars


In [16]:
exposures_entities_net.xs("EPBE",level=1)[[c for c in exposure_ent.columns if "ccy" not in c]]

,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_exposure_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity_ccy,,,,,,,,,,,
CHF,EUR,3.507185e+05,-4.712774e+06,-4.362055e+06,-1.470588e+05,-1.470588e+05,-294117.647059,4.565715e+06,LHS,RHS,4.067937e+06,RHS
EUR,EUR,6.923909e+07,-6.897756e+07,2.615338e+05,6.923909e+07,-6.897756e+07,261533.793454,0.000000e+00,NaN,NaN,0.000000e+00,NaN
USD,EUR,5.756789e+06,8.981906e+07,9.557585e+07,1.470588e+05,1.470588e+05,294117.647059,8.967200e+07,RHS,LHS,9.528173e+07,LHS
AUD,EUR,1.574927e+04,-4.826752e+05,-4.669259e+05,-1.470588e+05,-1.470588e+05,-294117.647059,3.356163e+05,LHS,RHS,1.728082e+05,RHS
AED,EUR,1.806255e+04,-1.438481e+05,-1.257856e+05,1.806255e+04,-1.438481e+05,-125785.575036,0.000000e+00,nan,nan,0.000000e+00,nan
PLN,EUR,2.871657e+06,-1.339182e+07,-1.052016e+07,-1.470588e+05,-1.470588e+05,-294117.647059,1.324476e+07,LHS,RHS,1.022604e+07,RHS
HKD,EUR,0.000000e+00,-2.467729e+03,-2.467729e+03,0.000000e+00,-2.467729e+03,-2467.729208,0.000000e+00,nan,nan,0.000000e+00,nan
CAD,EUR,7.970070e+03,1.165937e+05,1.245638e+05,7.970070e+03,1.165937e+05,124563.787083,0.000000e+00,nan,nan,0.000000e+00,nan
GBP,EUR,8.275989e+05,7.409683e+07,7.492443e+07,1.470588e+05,1.470588e+05,294117.647059,7.394978e+07,RHS,LHS,7.463032e+07,LHS


## Steps 2) & 4) (except EPUK)

In [18]:
exposures_entities_net_2 = pd.DataFrame()

for entity in exposures_entities_net.index.get_level_values(1).unique():

    exposure_ent = exposures_entities_net.xs(entity, level=1)

    if entity not in ("EPUK","EPL"):
        
        # If we don't have already exposure for the entity_ccy, we create it full of 0
        if exposure_ent.index.get_level_values(1).unique().item() not in exposure_ent.index.get_level_values(0):

            exposure_ent = pd.concat(
                [
                    exposure_ent,
                    pd.DataFrame(
                        np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.nan,np.nan,0,0,0,np.nan]).reshape(-1,1).T, 
                        index=pd.MultiIndex.from_tuples(
                            [(exposure_ent.index.get_level_values(1).unique().item(),exposure_ent.index.get_level_values(1).unique().item())], 
                            names=exposure_ent.index.names
                            ),
                        columns=exposure_ent.columns
                    )
                ]
            )
            
        # Locate the entity_ccy exposure to add the exposure from the internal changes (to the above and below) and from the hedges to EPUK to the above
        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_above_exposure_gbp",
            "new_above_exposure_local_ccy",
            "new_above_exposure_entity_ccy",
        ]] += (np.where(
                exposure_ent[["above_lhs_rhs"]]=="LHS",
                exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]],
                -1 * exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]]
        ).sum(axis=0) + np.where(
                exposure_ent[["above_to_epuk_lhs_rhs"]]=="LHS",
                exposure_ent[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]],
                -1 * exposure_ent[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]]
        ).sum(axis=0))

        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_below_exposure_gbp",
            "new_below_exposure_local_ccy",
            "new_below_exposure_entity_ccy",
        ]] += np.where(
                exposure_ent[["below_lhs_rhs"]]=="LHS",
                exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]],
                -1 * exposure_ent[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]]
        ).sum(axis=0)

        # The net exposure of the entity_ccy only changes due to the movement of exposure to EPUK
        exposure_ent.loc[
            exposure_ent.index.get_level_values(0)==exposure_ent.index.get_level_values(1).unique().item(),
        [
            "new_net_exposure_gbp",
            "new_net_exposure_local_ccy",
            "new_net_exposure_entity_ccy",
        ]] += np.where(
                exposure_ent[["above_to_epuk_lhs_rhs"]]=="LHS",
                exposure_ent[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]],
                -1 * exposure_ent[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]]
        ).sum(axis=0)



    exposure_ent["entity"] = entity
    exposure_ent = exposure_ent.reset_index().set_index(["currency","entity","entity_ccy"])

    exposures_entities_net_2 = pd.concat([exposures_entities_net_2,exposure_ent])



In [21]:
exposures_entities_net_2.xs("EPHK",level=1)

,,above_exposure_gbp,above_exposure_local_ccy,above_exposure_entity_ccy,below_exposure_gbp,below_exposure_local_ccy,below_exposure_entity_ccy,net_exposure_gbp,net_exposure_local_ccy,net_exposure_entity_ccy,new_above_exposure_gbp,...,new_net_exposure_entity_ccy,internal_change_gbp,internal_change_local_ccy,internal_change_entity_ccy,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_local_ccy,above_to_epuk_entity_ccy,above_to_epuk_lhs_rhs
currency,entity_ccy,,,,,,,,,,,,,,,,,,,,,
GBP,HKD,11136.29,11136.29,1.089778e+05,7.287082e+06,7287082.44,7.131010e+07,7.298219e+06,7298218.73,7.141907e+07,147058.823529,...,2.878183e+06,7.140024e+06,7.140024e+06,6.987100e+07,RHS,LHS,7.004101e+06,7.004101e+06,6.854089e+07,LHS
USD,HKD,-21877.70,-27373.43,-2.140917e+05,-3.578362e+06,-4477245.98,-3.501721e+07,-3.600239e+06,-4504619.41,-3.523130e+07,-147058.823529,...,-2.878183e+06,3.431303e+06,4.293246e+06,3.357812e+07,LHS,RHS,3.306122e+06,4.136619e+06,3.235312e+07,RHS
CHF,HKD,394.12,452.44,3.856786e+03,2.353128e+04,27013.35,2.302730e+05,2.392540e+04,27465.79,2.341298e+05,394.120000,...,2.341298e+05,0.000000e+00,0.000000e+00,0.000000e+00,nan,nan,0.000000e+00,0.000000e+00,0.000000e+00,nan
AUD,HKD,543.06,1047.10,5.314251e+03,2.760516e+04,53227.22,2.701392e+05,2.814822e+04,54274.32,2.754535e+05,543.060000,...,2.754535e+05,0.000000e+00,0.000000e+00,0.000000e+00,nan,nan,0.000000e+00,0.000000e+00,0.000000e+00,nan
AED,HKD,0.00,0.00,-2.980232e-08,3.557946e+04,163506.55,3.481742e+05,3.557946e+04,163506.55,3.481742e+05,0.000000,...,3.481742e+05,0.000000e+00,0.000000e+00,0.000000e+00,nan,nan,0.000000e+00,0.000000e+00,0.000000e+00,nan
EUR,HKD,628697.82,736266.25,6.152326e+06,-7.764403e+05,-909286.85,-7.598107e+06,-1.477425e+05,-173020.60,-1.445780e+06,-683.630463,...,-1.445780e+06,6.293815e+05,7.370667e+05,6.159015e+06,LHS,RHS,0.000000e+00,0.000000e+00,0.000000e+00,nan
PLN,HKD,0.00,0.00,0.000000e+00,3.332545e+04,168853.28,3.261169e+05,3.332545e+04,168853.28,3.261169e+05,0.000000,...,3.261169e+05,0.000000e+00,0.000000e+00,0.000000e+00,nan,nan,0.000000e+00,0.000000e+00,0.000000e+00,nan
HKD,HKD,18472.90,180772.24,1.807722e+05,-5.329014e+05,-5214878.25,-5.214878e+06,-5.144285e+05,-5034106.01,-5.034106e+06,637112.940463,...,3.115366e+07,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,NaN
CAD,HKD,0.00,0.00,0.000000e+00,-7.039829e+01,-121.10,-6.889051e+02,-7.039829e+01,-121.10,-6.889051e+02,0.000000,...,-6.889051e+02,0.000000e+00,0.000000e+00,0.000000e+00,nan,nan,0.000000e+00,0.000000e+00,0.000000e+00,nan


## Step 3)

In [26]:
exposures_entities_not_entity_ccy = exposures_entities_net_2.copy()
exposures_entities_not_entity_ccy = exposures_entities_not_entity_ccy[exposures_entities_not_entity_ccy.index.get_level_values(0) != exposures_entities_not_entity_ccy.index.get_level_values(2)]

In [27]:
exposures_to_epuk_entities_not_entity_ccy = exposures_entities_not_entity_ccy.reset_index()
exposures_to_epuk_entities_not_entity_ccy[
    [
        "above_to_epuk_gbp",
        "above_to_epuk_local_ccy",
        "above_to_epuk_entity_ccy"
    ]
] = np.where(
    exposures_to_epuk_entities_not_entity_ccy[["above_to_epuk_lhs_rhs"]] == "LHS",
    exposures_to_epuk_entities_not_entity_ccy[
        [
            "above_to_epuk_gbp",
            "above_to_epuk_local_ccy",
            "above_to_epuk_entity_ccy"
        ]
    ],
    -1 * exposures_to_epuk_entities_not_entity_ccy[
        [
            "above_to_epuk_gbp",
            "above_to_epuk_local_ccy",
            "above_to_epuk_entity_ccy"
        ]
    ]
)

# The change that we need to add by currency
exposures_to_epuk = exposures_to_epuk_entities_not_entity_ccy.groupby("currency")[[
            "above_to_epuk_gbp",
            "above_to_epuk_local_ccy",
            "above_to_epuk_entity_ccy"
]].sum().reset_index()

# Add these required fields
exposures_to_epuk[["entity","entity_ccy"]] = ["EPUK","GBP"]

exposures_to_epuk = exposures_to_epuk.set_index(["currency","entity","entity_ccy"])

# Add the exposures to EPUK (to the above and to the net)
exposures_entities_net_3 = exposures_entities_net_2.copy()
exposures_entities_net_3.loc[
    ((exposures_entities_net_2.index.get_level_values(1)=="EPUK")&(exposures_entities_net_2.index.get_level_values(2)=="GBP")),
    ["new_above_exposure_gbp","new_above_exposure_local_ccy","new_above_exposure_entity_ccy"]] += exposures_to_epuk.values
exposures_entities_net_3.loc[
    ((exposures_entities_net_2.index.get_level_values(1)=="EPUK")&(exposures_entities_net_2.index.get_level_values(2)=="GBP")),
    ["new_net_exposure_gbp","new_net_exposure_local_ccy","new_net_exposure_entity_ccy"]] += exposures_to_epuk.values

In [28]:
exposures_entities_net_3[exposures_entities_net_3.index.get_level_values(1)=="EPUK"][[c for c in exposure_ent.columns if "ccy" not in c]]

,,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_exposure_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity,entity_ccy,,,,,,,,,,,
CHF,EPUK,GBP,-4.355492e+06,3.180092e+06,-1.175400e+06,-6.302393e+06,3.180092e+06,-3.122301e+06,0.0,NaN,NaN,0.0,NaN
EUR,EPUK,GBP,-8.015238e+07,1.409674e+08,6.081498e+07,-7.977320e+07,1.409674e+08,6.119415e+07,0.0,NaN,NaN,0.0,NaN
USD,EPUK,GBP,-2.529958e+06,1.179007e+07,9.260114e+06,-2.529958e+06,1.179007e+07,9.260114e+06,0.0,NaN,NaN,0.0,NaN
PLN,EPUK,GBP,-9.311577e+05,5.686571e+05,-3.625006e+05,-4.999095e+06,5.686571e+05,-4.430438e+06,0.0,NaN,NaN,0.0,NaN
AED,EPUK,GBP,-2.200000e-01,-9.873862e+06,-9.873862e+06,-1.828469e+07,-9.873862e+06,-2.815855e+07,0.0,NaN,NaN,0.0,NaN
HKD,EPUK,GBP,3.183231e-12,-2.626989e+06,-2.626989e+06,7.669724e+07,-2.626989e+06,7.407025e+07,0.0,NaN,NaN,0.0,NaN
AUD,EPUK,GBP,-2.590651e+07,1.365527e+07,-1.225125e+07,-2.590651e+07,1.365527e+07,-1.225125e+07,0.0,NaN,NaN,0.0,NaN
CAD,EPUK,GBP,5.865700e+02,-3.635368e+06,-3.634782e+06,-1.022545e+07,-3.635368e+06,-1.386082e+07,0.0,NaN,NaN,0.0,NaN
GBP,EPUK,GBP,1.407771e+08,-2.020161e+08,-6.123892e+07,2.248089e+08,-2.020161e+08,2.279288e+07,0.0,NaN,NaN,0.0,NaN


## Step 4) (for EPUK)

In [29]:
# Here, we susbtract the amount instead of adding it
exposures_to_epuk_entity_ccy = (-1 * exposures_to_epuk_entities_not_entity_ccy.groupby(by=["entity_ccy"])[[
            "above_to_epuk_gbp",
            "above_to_epuk_local_ccy",
            "above_to_epuk_entity_ccy"
]].sum()).reset_index().rename(columns={"entity_ccy":"currency"})

exposures_to_epuk_entity_ccy[["entity","entity_ccy"]] = ["EPUK","GBP"]

exposures_to_epuk_entity_ccy = exposures_to_epuk_entity_ccy.set_index(["currency","entity","entity_ccy"])

# First we fill the gaps in currencies in EPUK
exposures_entities_net_4 = exposures_entities_net_3.copy()

for currency in exposures_to_epuk_entity_ccy.index.get_level_values(0):
    if currency not in exposures_entities_net_4.xs("EPUK",level=1).index.get_level_values(0):
        exposures_entities_net_4 = pd.concat(
                [
                    exposures_entities_net_4,
                    pd.DataFrame(
                        np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.nan,np.nan,0,0,0,np.nan]).reshape(-1,1).T, 
                        index=pd.MultiIndex.from_tuples(
                            [(currency,"EPUK","GBP")], 
                            names=exposures_entities_net_4.index.names
                            ),
                        columns=exposures_entities_net_4.columns
                    )
                ]
            )
        
# We need to fill also the the gaps for the exposures in the currencies that we are going to add
for currency in exposures_entities_net_4.xs("EPUK",level=1).index.get_level_values(0):
    if currency not in exposures_to_epuk_entity_ccy.index.get_level_values(0):
        exposures_to_epuk_entity_ccy = pd.concat(
                [
                    exposures_to_epuk_entity_ccy,
                    pd.DataFrame(
                        np.array([0,0,0]).reshape(-1,1).T, 
                        index=pd.MultiIndex.from_tuples(
                            [(currency,"EPUK","GBP")], 
                            names=exposures_to_epuk_entity_ccy.index.names
                            ),
                        columns=exposures_to_epuk_entity_ccy.columns
                    )
                ]
            )

# Add the exposures to EPUK (to the above and to the net)
exposures_to_epuk_entity_ccy.columns = ["new_above_exposure_gbp","new_above_exposure_local_ccy","new_above_exposure_entity_ccy"]
exposures_entities_net_4.loc[
    ((exposures_entities_net_4.index.get_level_values(1)=="EPUK")&(exposures_entities_net_4.index.get_level_values(2)=="GBP")),
    ["new_above_exposure_gbp","new_above_exposure_local_ccy","new_above_exposure_entity_ccy"]] += exposures_to_epuk_entity_ccy

exposures_to_epuk_entity_ccy.columns = ["new_net_exposure_gbp","new_net_exposure_local_ccy","new_net_exposure_entity_ccy"]
exposures_entities_net_4.loc[
    ((exposures_entities_net_4.index.get_level_values(1)=="EPUK")&(exposures_entities_net_4.index.get_level_values(2)=="GBP")),
    ["new_net_exposure_gbp","new_net_exposure_local_ccy","new_net_exposure_entity_ccy"]] += exposures_to_epuk_entity_ccy

In [30]:
exposures_entities_net_4.xs("EPUK",level=1)[[c for c in exposure_ent.columns if "ccy" not in c]]

,,above_exposure_gbp,below_exposure_gbp,net_exposure_gbp,new_above_exposure_gbp,new_below_exposure_gbp,new_net_exposure_gbp,internal_change_gbp,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_lhs_rhs
currency,entity_ccy,,,,,,,,,,,
CHF,GBP,-4.355492e+06,3.180092e+06,-1.175400e+06,-1.289268e+07,3.180092e+06,-9.712590e+06,0.0,NaN,NaN,0.0,NaN
EUR,GBP,-8.015238e+07,1.409674e+08,6.081498e+07,-2.352185e+08,1.409674e+08,-9.425111e+07,0.0,NaN,NaN,0.0,NaN
USD,GBP,-2.529958e+06,1.179007e+07,9.260114e+06,-1.652925e+06,1.179007e+07,1.013715e+07,0.0,NaN,NaN,0.0,NaN
PLN,GBP,-9.311577e+05,5.686571e+05,-3.625006e+05,-4.999095e+06,5.686571e+05,-4.430438e+06,0.0,NaN,NaN,0.0,NaN
AED,GBP,-2.200000e-01,-9.873862e+06,-9.873862e+06,-1.828469e+07,-9.873862e+06,-2.815855e+07,0.0,NaN,NaN,0.0,NaN
HKD,GBP,3.183231e-12,-2.626989e+06,-2.626989e+06,7.299926e+07,-2.626989e+06,7.037227e+07,0.0,NaN,NaN,0.0,NaN
AUD,GBP,-2.590651e+07,1.365527e+07,-1.225125e+07,-2.625503e+07,1.365527e+07,-1.259977e+07,0.0,NaN,NaN,0.0,NaN
CAD,GBP,5.865700e+02,-3.635368e+06,-3.634782e+06,-1.315992e+07,-3.635368e+06,-1.679529e+07,0.0,NaN,NaN,0.0,NaN
GBP,GBP,1.407771e+08,-2.020161e+08,-6.123892e+07,2.658462e+08,-2.020161e+08,6.383018e+07,0.0,NaN,NaN,0.0,NaN


In [31]:
exposures_entities.net_exposure_gbp.sum()

10725717.776620379

In [32]:
exposures_entities_net_4.net_exposure_gbp.sum()

10725717.77662034

In [33]:
exposures_entities_net_4.new_net_exposure_gbp.sum()

10725717.77662034

## Step 5) & 6)

In [34]:
exposures_entities_net_5 = exposures_entities_net_4.copy().xs("EPUK",level=1)

In [35]:
import warnings

warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

entity = "EPUK"

exposures_entities_net_5[
    [
        'new_above_exposure_gbp', 
        'new_above_exposure_local_ccy', 
        'new_above_exposure_entity_ccy', 
        'new_below_exposure_gbp', 
        'new_below_exposure_local_ccy', 
        'new_below_exposure_entity_ccy', 
        'new_net_exposure_gbp', 
        'new_net_exposure_local_ccy', 
        'new_net_exposure_entity_ccy', 
        'internal_change_gbp', 
        'internal_change_local_ccy', 
        'internal_change_entity_ccy', 
        'above_lhs_rhs', 
        'below_lhs_rhs', 
        'above_to_epuk_gbp', 
        'above_to_epuk_local_ccy', 
        'above_to_epuk_entity_ccy', 
        'above_to_epuk_lhs_rhs'
    ]
] = exposures_entities_net_5.apply(
    lambda row: pd.Series(
        net_exposure(
            row.name[0],
            entity, 
            row.name[1], 
            row['new_above_exposure_gbp'], 
            row['new_above_exposure_local_ccy'], 
            row['new_above_exposure_entity_ccy'], 
            row['new_below_exposure_gbp'], 
            row['new_below_exposure_local_ccy'], 
            row['new_below_exposure_entity_ccy'], 
            entity_threshold,
            exclude_epuk=False,
            exclude_epl=True
        )
    ),
    axis=1
)


In [36]:
exposures_entities_net_5

,,above_exposure_gbp,above_exposure_local_ccy,above_exposure_entity_ccy,below_exposure_gbp,below_exposure_local_ccy,below_exposure_entity_ccy,net_exposure_gbp,net_exposure_local_ccy,net_exposure_entity_ccy,new_above_exposure_gbp,...,new_net_exposure_entity_ccy,internal_change_gbp,internal_change_local_ccy,internal_change_entity_ccy,above_lhs_rhs,below_lhs_rhs,above_to_epuk_gbp,above_to_epuk_local_ccy,above_to_epuk_entity_ccy,above_to_epuk_lhs_rhs
currency,entity_ccy,,,,,,,,,,,,,,,,,,,,,
CHF,GBP,-4.355492e+06,-5.000000e+06,-4.355492e+06,3.180092e+06,3.650669e+06,3.180092e+06,-1.175400e+06,-1349330.75,-1.175400e+06,-1.470588e+05,...,-1.464266e+06,3.033033e+06,3.481849e+06,3.033033e+06,RHS,LHS,9.712590e+06,1.114982e+07,9.712590e+06,RHS
EUR,GBP,-8.015238e+07,-9.386620e+07,-8.015238e+07,1.409674e+08,1.650864e+08,1.409674e+08,6.081498e+07,71220233.51,6.081498e+07,-1.470588e+05,...,-2.662578e+07,1.408203e+08,1.649142e+08,1.408203e+08,RHS,LHS,9.425111e+07,1.103772e+08,9.425111e+07,RHS
USD,GBP,-2.529958e+06,-3.165483e+06,-2.529958e+06,1.179007e+07,1.475174e+07,1.179007e+07,9.260114e+06,11586254.88,9.260114e+06,1.470588e+05,...,5.144280e+05,1.164301e+07,1.456774e+07,1.164301e+07,RHS,LHS,9.843030e+06,1.231560e+07,9.843030e+06,LHS
PLN,GBP,-9.311577e+05,-4.717987e+06,-9.311577e+05,5.686571e+05,2.881270e+06,5.686570e+05,-3.625006e+05,-1836717.34,-3.625007e+05,-1.470588e+05,...,-6.960118e+05,4.215982e+05,2.136153e+06,4.215982e+05,RHS,LHS,4.430438e+06,2.244813e+07,4.430438e+06,RHS
AED,GBP,-2.200000e-01,-1.000000e+00,-2.176026e-01,-9.873862e+06,-4.537565e+07,-9.873862e+06,-9.873862e+06,-45375654.89,-9.873862e+06,-1.470588e+05,...,9.158068e+05,9.726803e+06,4.469984e+07,9.726803e+06,LHS,RHS,2.786443e+07,1.280519e+08,2.786443e+07,RHS
HKD,GBP,3.183231e-12,0.000000e+00,1.364242e-12,-2.626989e+06,-2.570725e+07,-2.626989e+06,-2.626989e+06,-25707249.80,-2.626989e+06,1.470588e+05,...,3.920820e+07,2.479930e+06,2.426816e+07,2.479930e+06,LHS,RHS,7.037227e+07,6.886506e+08,7.037227e+07,LHS
AUD,GBP,-2.590651e+07,-4.995195e+07,-2.590651e+07,1.365527e+07,2.632956e+07,1.365527e+07,-1.225125e+07,-23622390.23,-1.225125e+07,-1.470588e+05,...,-3.234793e+05,1.350821e+07,2.604601e+07,1.350821e+07,RHS,LHS,1.259977e+07,2.429439e+07,1.259977e+07,RHS
CAD,GBP,5.865700e+02,1.009030e+03,5.865729e+02,-3.635368e+06,-6.253606e+06,-3.635368e+06,-3.634782e+06,-6252596.53,-3.634782e+06,-1.470588e+05,...,-4.157201e+06,3.488310e+06,6.000633e+06,3.488310e+06,LHS,RHS,1.650117e+07,2.838552e+07,1.650117e+07,RHS
GBP,GBP,1.407771e+08,1.407771e+08,1.407771e+08,-2.020161e+08,-2.020161e+08,-2.020161e+08,-6.123892e+07,-61238916.94,-6.123892e+07,2.658462e+08,...,5.579734e+07,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,NaN


### Step 5.1) Add the entity_ccy amount of the back-to-back from the internal and also the ones from the external

In [37]:
# Above
exposures_entities_net_5.loc[
    exposures_entities_net_5.index.get_level_values(0)=="GBP",
    ["new_above_exposure_gbp","new_above_exposure_local_ccy","new_above_exposure_entity_ccy"]
] += (np.where(
    exposures_entities_net_5[["above_lhs_rhs"]]=="LHS",
    exposures_entities_net_5[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]],
    -1 * exposures_entities_net_5[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]]
).sum(axis=0) + np.where(
    exposures_entities_net_5[["above_to_epuk_lhs_rhs"]]=="LHS",
    exposures_entities_net_5[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]],
    -1 * exposures_entities_net_5[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]]
).sum(axis=0))

# Below
exposures_entities_net_5.loc[
    exposures_entities_net_5.index.get_level_values(0)=="GBP",
    ["new_below_exposure_gbp","new_below_exposure_local_ccy","new_below_exposure_entity_ccy"]
] += (np.where(
    exposures_entities_net_5[["below_lhs_rhs"]]=="LHS",
    exposures_entities_net_5[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]],
    -1 * exposures_entities_net_5[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]]
).sum(axis=0))

# Net (only changes due to the external movement of exposure)
exposures_entities_net_5.loc[
    exposures_entities_net_5.index.get_level_values(0)=="GBP",
    ["new_net_exposure_gbp","new_net_exposure_local_ccy","new_net_exposure_entity_ccy"]
] += np.where(
    exposures_entities_net_5[["above_to_epuk_lhs_rhs"]]=="LHS",
    exposures_entities_net_5[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]],
    -1 * exposures_entities_net_5[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]]
).sum(axis=0)


# Reshape it to the standard shape
exposures_entities_net_5 = exposures_entities_net_5.reset_index()
exposures_entities_net_5["entity"] = "EPUK"

exposures_entities_net_5 = exposures_entities_net_5.set_index(["currency","entity","entity_ccy"]) 

Put all together

In [38]:
exposures_entities_net_6 = pd.concat([exposures_entities_net_4[exposures_entities_net_4.index.get_level_values(1)!="EPUK"],exposures_entities_net_5])

## Step 7

In [39]:
exposures_epl = exposures_entities_net_6[exposures_entities_net_6.index.get_level_values(1)=="EPL"]

In [40]:
exposures_entities_currency_to_epl = exposures_entities_net_6[exposures_entities_net_6.index.get_level_values(0)==exposures_entities_net_6.index.get_level_values(2)].reset_index()
exposures_entities_currency_to_epl["entity"] = "EPL"
exposures_entities_currency_to_epl["entity_ccy"] = "GBP"
exposures_entities_currency_to_epl = exposures_entities_currency_to_epl.set_index(["currency","entity","entity_ccy"]).groupby(["currency","entity","entity_ccy"]).sum()
exposures_entities_currency_to_epl[["above_lhs_rhs","below_lhs_rhs","above_to_epuk_lhs_rhs"]] = np.nan

exposures_entities_currency_to_epl = exposures_entities_currency_to_epl[exposures_entities_net_6.columns]

First, we fill the currencies that we currently don't have in EPL but that we are going to move there

In [ ]:
for currency in exposures_entities_currency_to_epl.index.get_level_values(0).unique():
    if currency not in exposures_epl.index.get_level_values(0).unique():

        exposures_epl = pd.concat(
            [
                exposures_epl,
                pd.DataFrame(
                    np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.nan,np.nan,0,0,0,np.nan]).reshape(-1,1).T, 
                    index=pd.MultiIndex.from_tuples(
                        [(currency,"EPL","GBP")], 
                        names=exposures_epl.index.names
                        ),
                    columns=exposures_epl.columns
                )
            ]
        )

for currency in exposures_epl.index.get_level_values(0).unique():
    if currency not in exposures_entities_currency_to_epl.index.get_level_values(0).unique():

        exposures_entities_currency_to_epl = pd.concat(
            [
                exposures_entities_currency_to_epl,
                pd.DataFrame(
                    np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,np.nan,np.nan,0,0,0,np.nan]).reshape(-1,1).T, 
                    index=pd.MultiIndex.from_tuples(
                        [(currency,"EPL","GBP")], 
                        names=exposures_entities_currency_to_epl.index.names
                        ),
                    columns=exposures_entities_currency_to_epl.columns
                )
            ]
        )

Add (fictitiously) the exposures of each entity_ccy to EPL

In [ ]:
exposures_epl[exposures_epl.columns[9:]] = exposures_epl[exposures_epl.columns[9:]] + exposures_entities_currency_to_epl[exposures_entities_currency_to_epl.columns[9:]]

In [ ]:
exposures_epl.xs("HKD",level=0).new_below_exposure_entity_ccy / exposures_epl.xs("HKD",level=0).new_below_exposure_gbp

In [ ]:
exposures_epl.new_net_exposure_gbp.sum()

In [ ]:
exposures_epl

Net between above and below in EPL

In [ ]:
import warnings

warnings.filterwarnings('ignore', category=pd.core.common.SettingWithCopyWarning)

entity = "EPL"

exposures_epl[
    [
        'new_above_exposure_gbp', 
        'new_above_exposure_local_ccy', 
        'new_above_exposure_entity_ccy', 
        'new_below_exposure_gbp', 
        'new_below_exposure_local_ccy', 
        'new_below_exposure_entity_ccy', 
        'new_net_exposure_gbp', 
        'new_net_exposure_local_ccy', 
        'new_net_exposure_entity_ccy', 
        'internal_change_gbp', 
        'internal_change_local_ccy', 
        'internal_change_entity_ccy', 
        'above_lhs_rhs', 
        'below_lhs_rhs', 
        'above_to_epuk_gbp', 
        'above_to_epuk_local_ccy', 
        'above_to_epuk_entity_ccy', 
        'above_to_epuk_lhs_rhs'
    ]
] = exposures_epl.apply(
    lambda row: pd.Series(
        net_exposure(
            row.name[0],
            entity, 
            row.name[1], 
            row['new_above_exposure_gbp'], 
            row['new_above_exposure_local_ccy'], 
            row['new_above_exposure_entity_ccy'], 
            row['new_below_exposure_gbp'], 
            row['new_below_exposure_local_ccy'], 
            row['new_below_exposure_entity_ccy'], 
            entity_threshold,
            exclude_epuk=True,
            exclude_epl=False
        )
    ),
    axis=1
)


In [ ]:
exposures_epl.new_net_exposure_gbp.sum()

### Step 7.1) Add the entity_ccy amount of the back-to-back from the internal and also the ones from the external

In [ ]:
# Above
exposures_epl.loc[
    exposures_epl.index.get_level_values(0)=="GBP",
    ["new_above_exposure_gbp","new_above_exposure_local_ccy","new_above_exposure_entity_ccy"]
] += (np.where(
    exposures_epl[["above_lhs_rhs"]]=="LHS",
    exposures_epl[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]],
    -1 * exposures_epl[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]]
).sum(axis=0) + np.where(
    exposures_epl[["above_to_epuk_lhs_rhs"]]=="LHS",
    exposures_epl[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]],
    -1 * exposures_epl[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]]
).sum(axis=0))

# Below
exposures_epl.loc[
    exposures_epl.index.get_level_values(0)=="GBP",
    ["new_below_exposure_gbp","new_below_exposure_local_ccy","new_below_exposure_entity_ccy"]
] += (np.where(
    exposures_epl[["below_lhs_rhs"]]=="LHS",
    exposures_epl[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]],
    -1 * exposures_epl[["internal_change_gbp","internal_change_local_ccy","internal_change_entity_ccy"]]
).sum(axis=0))

# Net (only changes due to the external movement of exposure)
exposures_epl.loc[
    exposures_epl.index.get_level_values(0)=="GBP",
    ["new_net_exposure_gbp","new_net_exposure_local_ccy","new_net_exposure_entity_ccy"]
] += np.where(
    exposures_epl[["above_to_epuk_lhs_rhs"]]=="LHS",
    exposures_epl[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]],
    -1 * exposures_epl[["above_to_epuk_gbp","above_to_epuk_local_ccy","above_to_epuk_entity_ccy"]]
).sum(axis=0)


# Reshape it to the standard shape
exposures_epl = exposures_epl.reset_index()
exposures_epl["entity"] = "EPL"

exposures_epl = exposures_epl.set_index(["currency","entity","entity_ccy"]) 

In [ ]:
exposures_epl.new_net_exposure_gbp.sum()

In [ ]:
exposures_epl.sort_values("above_to_epuk_local_ccy")

# AHORA SOLO FALTA AÑADIR LAS EXPOSICIONES AL TOTAL, COMPROBAR QUE TODO ME QUEDE NETEADO, Y MOVER LOS HEDGES EXTERNOS DE EPL A EPUK

## Step 9) List all the hedges that take place:

In [ ]:
hedges = pd.DataFrame()

The internal hedges for each entity (except EPUK and EPL)

In [ ]:
print("Internal hedges: \n")
count = 0
for row in exposures_entities_net_6.iterrows():
    
    if row[1]["internal_change_gbp"] != 0 and row[0][1] not in ("EPUK","EPL"):
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "amount_local_ccy": row[1]['internal_change_local_ccy'],
                "amount_entity_ccy": row[1]['internal_change_entity_ccy'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "entities_above_below"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['below_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "amount_local_ccy": row[1]['internal_change_local_ccy'],
                "amount_entity_ccy": row[1]['internal_change_entity_ccy'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "BS FX Risk hedge",
                "type": "entities_above_below"
            },
            orient='index').T
        ])
        # print(f"{count}.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['above_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['below_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: BS FX Risk hedge.")
        # print()
        

In [ ]:
hedges.shape

The hedges to EPUK for each entity (except EPUK and EPL)

In [ ]:
print("Internal hedges: \n")
count = 0
for row in exposures_entities_net_6.iterrows():
    
    if row[1]["above_to_epuk_gbp"] != 0 and row[0][1] not in ("EPUK","EPL"):
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_to_epuk_lhs_rhs'],
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "amount_local_ccy": row[1]['above_to_epuk_local_ccy'],
                "amount_entity_ccy": row[1]['above_to_epuk_entity_ccy'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "above_to_epuk"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": np.where(row[1]['above_to_epuk_lhs_rhs']=='RHS','LHS','RHS'),
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "amount_local_ccy": row[1]['above_to_epuk_local_ccy'],
                "amount_entity_ccy": row[1]['above_to_epuk_entity_ccy'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "above_to_epuk"
            },
            orient='index').T
        ])
        # print(f"{count}.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['above_to_epuk_lhs_rhs']}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print(f"{row[0][0]}{row[0][2]} {np.where(row[1]['above_to_epuk_lhs_rhs']=='RHS','LHS','RHS')}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: EPUK. Dealset: Revenue FX Risk hedge.")
        # print()
        

In [ ]:
hedges.shape

The internal hedges in EPUK

In [ ]:
print("Internal hedges: \n")
count = 0
for row in exposures_entities_net_6.iterrows():
    
    if row[1]["internal_change_gbp"] != 0 and row[0][1] == "EPUK":
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "amount_local_ccy": row[1]['internal_change_local_ccy'],
                "amount_entity_ccy": row[1]['internal_change_entity_ccy'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "epuK_above_below"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['below_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "amount_local_ccy": row[1]['internal_change_local_ccy'],
                "amount_entity_ccy": row[1]['internal_change_entity_ccy'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "BS FX Risk hedge",
                "type": "epuK_above_below"
            },
            orient='index').T
        ])
        # print(f"{count}.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['above_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['below_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: BS FX Risk hedge.")
        # print()
        

In [ ]:
hedges.shape

The external hedges in EPUK for covering EPUK (except the entity ccys)

In [ ]:
print("External hedges: \n")
count = 0
for row in exposures_entities_net_6.iterrows():
    
    if row[1]["above_to_epuk_gbp"] != 0 and row[0][1] == "EPUK":
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "external",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_to_epuk_lhs_rhs'],
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "amount_local_ccy": row[1]['above_to_epuk_local_ccy'],
                "amount_entity_ccy": row[1]['above_to_epuk_entity_ccy'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "external_epuk"
            },
            orient='index').T
        ])

        # print(f"{count}.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['above_to_epuk_lhs_rhs']}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print()
        

In [ ]:
hedges.shape

The internal hedges in EPL to cover the entity curencies

In [ ]:
print("Internal hedges: \n")
count = 0
for row in exposures_epl.iterrows():
    
    if row[1]["internal_change_gbp"] != 0:
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "amount_local_ccy": row[1]['internal_change_local_ccy'],
                "amount_entity_ccy": row[1]['internal_change_entity_ccy'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "epl_above_below_entity_currencies"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['below_lhs_rhs'],
                "amount_gbp": row[1]['internal_change_gbp'],
                "amount_local_ccy": row[1]['internal_change_local_ccy'],
                "amount_entity_ccy": row[1]['internal_change_entity_ccy'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "BS FX Risk hedge",
                "type": "epl_above_below_entity_currencies"
            },
            orient='index').T
        ])
        # print(f"{count}.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['above_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print(f"{row[0][0]}{row[0][2]} {row[1]['below_lhs_rhs']}. Amount: {round(row[1]['internal_change_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: BS FX Risk hedge.")
        # print()
        

The external hedges in EPL (booked by EPUK) and the corresponding movement from EPUK to EPL

In [ ]:
print("External & internal hedges: \n")
count = 0
for row in exposures_epl.iterrows():
    
    if row[1]["above_to_epuk_gbp"] != 0:
        count += 1
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "external",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_to_epuk_lhs_rhs'],
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "amount_local_ccy": row[1]['above_to_epuk_local_ccy'],
                "amount_entity_ccy": row[1]['above_to_epuk_entity_ccy'],
                "entity": "EPUK",
                "entity_ccy": "GBP",
                "dealset": "Revenue FX Risk hedge",
                "type": "external_epl_in_epuk"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": np.where(row[1]['above_to_epuk_lhs_rhs']=='RHS','LHS','RHS'),
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "amount_local_ccy": row[1]['above_to_epuk_local_ccy'],
                "amount_entity_ccy": row[1]['above_to_epuk_entity_ccy'],
                "entity": "EPUK",
                "entity_ccy": "GBP",
                "dealset": "Revenue FX Risk hedge",
                "type": "internal_epuk_epl"
            },
            orient='index').T
        ])
        hedges = pd.concat([hedges,pd.DataFrame.from_dict(
            {
                "internal_external": "internal",
                "pair": f"{row[0][0]}{row[0][2]}",
                "direction": row[1]['above_to_epuk_lhs_rhs'],
                "amount_gbp": row[1]['above_to_epuk_gbp'],
                "amount_local_ccy": row[1]['above_to_epuk_local_ccy'],
                "amount_entity_ccy": row[1]['above_to_epuk_entity_ccy'],
                "entity": row[0][1],
                "entity_ccy": row[0][2],
                "dealset": "Revenue FX Risk hedge",
                "type": "internal_epuk_epl"
            },
            orient='index').T
        ])
        # print(f"{count}.")
        # print(f"(External) {row[0][0]}{row[0][2]} {row[1]['above_to_epuk_lhs_rhs']}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: EPUK. Dealset: Revenue FX Risk hedge.")
        # print(f"(Internal) {row[0][0]}{row[0][2]} {np.where(row[1]['above_to_epuk_lhs_rhs']=='RHS','LHS','RHS')}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: EPUK. Dealset: Revenue FX Risk hedge.")
        # print(f"(Internal) {row[0][0]}{row[0][2]} {row[1]['above_to_epuk_lhs_rhs']}. Amount: {round(row[1]['above_to_epuk_gbp']):.3e} GBP. Entity: {row[0][1]}. Dealset: Revenue FX Risk hedge.")
        # print()
        

In [ ]:
hedges = hedges.reset_index(drop=True)

In [ ]:
hedges

In [ ]:
exposures_entities_net_6.loc[
    (np.abs(exposures_entities_net_6.new_above_exposure_gbp)>2*entity_threshold+1)|
    (np.abs(exposures_entities_net_6.new_below_exposure_gbp)>2*entity_threshold+1)|
    (np.abs(exposures_entities_net_6.new_net_exposure_gbp)>2*entity_threshold+1)
][[c for c in exposures_entities_net_6.columns if "ccy" not in c]]

In [ ]:
def get_config():
    config = ConfigParser()
    config.read(os.path.join("C:/Users/andres.mireles_ebury/Desktop/Projects/FX Exposure/fx_exposure/config.ini"))

    return config


In [ ]:
config = get_config()
bq_client, _ = get_bq_link(config)

In [ ]:
hedges.sort_values("amount_local_ccy")

In [ ]:
hedges["direction"] = hedges["direction"].astype(str)
hedges["amount_gbp"] = hedges["amount_gbp"].astype(float)
hedges["amount_local_ccy"] = hedges["amount_local_ccy"].astype(float)
hedges["amount_entity_ccy"] = hedges["amount_entity_ccy"].astype(float)

In [ ]:
hedges.dtypes

In [ ]:
import upload

upload.upload_table(
    hedges,
    "hedges_entities",
    bq_client,
    config,
    balance_date
)

In [ ]:
# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(1,2,figsize=(15,5))

# axs[0].bar(x=exposures_entities["above_exposure_gbp"].index.get_level_values(0),height=exposures_entities["above_exposure_gbp"])
# axs[0].bar(x=exposures_entities["below_exposure_gbp"].index.get_level_values(0),height=exposures_entities["below_exposure_gbp"])
# axs[0].legend(["Above Exposure (GBP)","Below Exposure (GBP)"])
# axs[0].set_title("Pre Exposure")
# axs[0].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)

# axs[1].bar(x=exposures_entities["new_above_gbp"].index.get_level_values(0),height=exposures_entities["new_above_gbp"])
# axs[1].bar(x=exposures_entities["new_below_gbp"].index.get_level_values(0),height=exposures_entities["new_below_gbp"])
# axs[1].legend(["New Above Exposure (GBP)","New Below Exposure (GBP)"])
# axs[1].set_title("New Exposure")
# axs[1].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)
# plt.tight_layout()

In [ ]:
# import matplotlib.pyplot as plt

# fig, axs = plt.subplots(1,2,figsize=(15,5))

# axs[0].bar(x=exposures_entities["net_exposure_gbp"].index.get_level_values(0),height=exposures_entities["net_exposure_gbp"],color="green")
# axs[0].legend(["Net Exposure (GBP)"])
# axs[0].set_title("Pre Exposure")
# axs[0].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)

# axs[1].bar(x=exposures_entities["new_net_gbp"].index.get_level_values(0),height=exposures_entities["new_net_gbp"],color="green")
# axs[1].legend(["New Net Exposure (GBP)"])
# axs[1].set_title("New Exposure")
# axs[1].set_xticklabels(exposures_entities.index.get_level_values(0), rotation=45)
# plt.show()